# Imports   

In [2]:
import os
import gym
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from colabgymrender.recorder import Recorder

# Load Enviorment

In [3]:
enviorment_name = "ALE/Breakout-v5"
env = gym.make(enviorment_name)
env = Recorder(env, './video')

In [4]:
# Test the enviorment
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:4.0
Episode:2 Score:4.0
Episode:3 Score:0.0
Episode:4 Score:1.0
Episode:5 Score:3.0


In [6]:
env = make_atari_env(enviorment_name, n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [7]:
log_path = os.path.join('Training', 'Logs')
a2c_model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [15]:
# train the a2c model
a2c_model.learn(total_timesteps=100000)

Logging to Training\Logs\A2C_2
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 237      |
|    ep_rew_mean        | 1.93     |
| time/                 |          |
|    fps                | 193      |
|    iterations         | 100      |
|    time_elapsed       | 10       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.345    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.0844  |
|    value_loss         | 0.21     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 237      |
|    ep_rew_mean        | 1.86     |
| time/                 |          |
|    fps                | 195      |
|    iterations         | 200      |
|    time_elapsed       | 20       |
|    total_timesteps    | 4000     |
| train

# Save the Model

In [20]:
a2c_path = os.path.join('Training', 'saved_models', 'A2C_Breakout_Model')
a2c_model.save(a2c_path)

# Evaluation

In [21]:
# create a single enviorment obj
env = make_atari_env(enviorment_name, n_envs=1, seed=0)
# keep the num of stack as 4 so the evaluation will success
env = VecFrameStack(env, n_stack=4)

In [25]:
evaluate_policy(a2c_model, env, n_eval_episodes=10, render=True)

(16.5, 2.5787593916455256)

In [26]:
env.close()

# Trying different algorithm - PPO

In [28]:
env = make_atari_env(enviorment_name, n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

In [30]:
# create ppo model instace
ppo_model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [31]:
# train the model
ppo_model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 235      |
|    ep_rew_mean     | 1.76     |
| time/              |          |
|    fps             | 227      |
|    iterations      | 1        |
|    time_elapsed    | 36       |
|    total_timesteps | 8192     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 257        |
|    ep_rew_mean          | 2.34       |
| time/                   |            |
|    fps                  | 139        |
|    iterations           | 2          |
|    time_elapsed         | 117        |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.02326653 |
|    clip_fraction        | 0.164      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.37      |
|    explained_variance   | -0.0654    |
|    learning_

In [32]:
ppo_path = os.path.join('Training', 'saved_models', 'PPO_model_Breakout2')
ppo_model.save(ppo_path)

In [33]:
# create a single enviorment obj
env = make_atari_env(enviorment_name, n_envs=1, seed=0)
# keep the num of stack as 4 so the evaluation will success
env = VecFrameStack(env, n_stack=4)

In [34]:
evaluate_policy(ppo_model, env, n_eval_episodes=10, render=True)

(15.7, 3.257299494980466)

In [35]:
env.close()

# Conclusion

After Trying both A2C and PPO Model-free algorithms on the Atari enviorment with a speed up of x4 <br>
with total timestamps of 100,000 each, we can see that the difference isn't big between the result. <br>

A2C - 16.5 points per game on average <br>
PPO - 15.7 point per game on average <br>